# 教師あり学習 (2)

長くなってきたので章を分けました。






## アルゴリズム5 決定木のアンサンブル法

- アンサンブル法 (Ensembles): 複数の機械学習モデルを組合せてより強力なモデルを構築する手法。
- ランダムフォレストと勾配ブースティングが有名。

### ランダムフォレスト

- 決定木の過剰適合しやすいという欠点に対する対処法の一つ。
- **少しずつ異なる決定木**をたくさん作って、その平均をとるという手法。
    - 木が沢山あるので森

#### ランダムフォレストの構築

- **少しずつ異なる決定木**は次のように作成する。
    - 元のデータセットから**ブートストラップサンプリング**(復元抽出)により新しいデータセット(ブートストラップサンプル)を作成する。
    - 新しいデータセットから決定木を作る。このとき、そのままではブートストラップサンプルから生成される決定木の相関が高くなるので、さらに一工夫することで木の多様性を高める。
      - 分割に使う特徴量を選択する前に、$d$個の特徴量からランダムに$d'$個の特徴量を選ぶ。
          - $d'=\lfloor\sqrt{d}\rfloor$が推奨されているが問題によって調整する必要がある。
          - sklearnでは`max_features=`で指定する。
          - `max_features=`が大きいと木の間の相関が高まり、訓練データに適合しやすくなる。
          - `max_features=`が小さいと木の間の相関は減るが、深いところまで探索しないと訓練データに適合できなくなる。
      - 選択された$d'$個の特徴量から最適な分割を与える特徴量と分割点を定める。
      - 上記の手順を全てのノードで実行する。
- 予測の際は全ての決定木に対して予測する。
    - 回帰のとき: 平均値を予測結果とする。
    - クラス分類のとき: ソフト投票(soft voting: 各木の予測確率の平均値が最大のクラスを予測クラスとする)でクラスを決める。
        - ソフト投票に対するハード投票(hard voting)は、各木に対する予測クラスを先に求め、その多数決で決めるもの。
        
#### ランダムフォレストの解析

**two_moons**データセットにランダムフォレストを適用する。


```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=100, noise=0.25, random_state=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest.fit(X_train, y_train)
```

それぞれの決定木は`estimator_`に格納されているので、最終的な予測とともに可視化してみよう。


```python
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
for i, (ax, tree) in enumerate(zip(axes.ravel(), forest.estimators_)):
  ax.set_title("Tree {}".format(i))
  mglearn.plots.plot_tree_partition(X_train, y_train, tree, ax=ax)
  
mglearn.plots.plot_2d_separator(forest, X_train, fill=True, ax=axes[-1, -1], alpha=.4)
axes[-1, -1].set_title("ランダムフォレスト")
mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train)
```

![](png2/unnamed-chunk-4-1.png)<!-- -->

もう一つの例としてcancerにランダムフォレストを適用してみよう。今回は木の数は100とする。


```python
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
  cancer.data, cancer.target, random_state=0
)
forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(X_train, y_train)
print("訓練セットの精度: {:.3f}".format(forest.score(X_train, y_train)))
 ## 訓練セットの精度: 1.000
print("テストセットの精度: {:.3f}".format(forest.score(X_test, y_test)))
 ## テストセットの精度: 0.972
```

ランダムフォレストはチューニングをしなくてもデフォルトのパラメータで十分に高い精度を出すことが多い。

複数の木の平均として求めるため、特徴量の重要度の信頼性も高い。


```python
n_features = cancer.data.shape[1]
plt.barh(range(n_features), forest.feature_importances_, align='center')
plt.yticks(np.arange(n_features), cancer.feature_names)
plt.xlabel("特徴量の重要度")
plt.ylabel("特徴量")
```

![](png2/unnamed-chunk-7-1.png)<!-- -->

#### 長所、短所、パラメータ

- ランダムフォレストは現在最も広く使われている機械学習手法の一つ。
    - 強力で、チューニングをさほど必要とせず、スケール変換も不要。
- 複数の木を合成する関係上、モデルの解釈は決定木より難しくなる。簡単な解釈が欲しければ決定木を使うのも手。
- 個々の木を作るプロセスは独立なので、並列化も容易にできる。使用コア数は`n_jobs`で指定し、-1を指定すれば全てのコアを使う。
- 高次元で疎なデータに対しては上手く機能しない事が多い。線形モデルを使おう。
- 訓練にも予測にも時間がかかり、メモリも多く使う。速度やメモリが重要な時は線形モデルを使おう。
- `n_estimatros`と`max_features`、`max_depth`などの枝刈りパラメータは検討の余地がある。`n_estimator`は時間とメモリの許す限り大きくすべきだ。
- `max_features`はデフォルトでは以下の設定になっており、大抵の場合はデフォルトでも問題はない。
    - クラス分類: `sqrt(n_features)`
    - 回帰: `n_features`
- `max_features`や`max_leaf_nodes`を設定すると性能が上がったり、訓練と予測の時間が縮まることもある。

### 勾配ブースティング回帰木(勾配ブースティングマシン)

- 1つ前の決定木を次の決定木で修正するように学習が進む。
- デフォルトでは乱数性は無い。
- 深さ1〜5の浅い決定木が使われる。これによりメモリと計算時間を節約できる。
- **弱学習器**(weak learner)を複数組み合わせる。
    - ひとつの学習器は限られた範囲で良い予測を与えるだけだが、組み合わせることで性能を向上させる。
- ランダムフォレストよりパラメータの影響をうけるが、きちんとチューニングをすればランダムフォレストを上回る性能を出せる。
- 重要なパラメータとして**learning_rate**(学習率)がある。これは個々の決定木がどの程度補正を行うのかの強さを決定するパラメータで、大きいほど補正が強い = モデルが複雑になる。

cancerデータで例を見る。


```python
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(
  cancer.data, cancer.target, random_state=0
)
gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train)
print("訓練セットに対する精度: {:.3f}".format(gbrt.score(X_train, y_train)))
 ## 訓練セットに対する精度: 1.000
print("テストセットに対する精度: {:.3f}".format(gbrt.score(X_test, y_test)))
 ## テストセットに対する精度: 0.958
```

過剰適合が疑われる(訓練セットの精度が高すぎる)ので、モデルを単純にする。


```python
## 枝刈りの深さを浅くする
gbrt = GradientBoostingClassifier(random_state=0, max_depth=1)
gbrt.fit(X_train, y_train)
print("訓練セットに対する精度: {:.3f}".format(gbrt.score(X_train, y_train)))
 ## 訓練セットに対する精度: 0.991
print("テストセットに対する精度: {:.3f}".format(gbrt.score(X_test, y_test)))
 ## テストセットに対する精度: 0.972
```


```python
## 学習率を下げる
gbrt = GradientBoostingClassifier(random_state=0, learning_rate=.01)
gbrt.fit(X_train, y_train)
print("訓練セットに対する精度: {:.3f}".format(gbrt.score(X_train, y_train)))
 ## 訓練セットに対する精度: 0.988
print("テストセットに対する精度: {:.3f}".format(gbrt.score(X_test, y_test)))
 ## テストセットに対する精度: 0.965
```

この例では枝刈りを強くしたほうが汎化性能が上がった。パラメータを可視化してみる。


```python
gbrt = GradientBoostingClassifier(random_state=0, max_depth=1)
gbrt.fit(X_train, y_train)
n_features = cancer.data.shape[1]
plt.barh(range(n_features), gbrt.feature_importances_, align="center")
plt.yticks(np.arange(n_features), cancer.feature_names)
plt.xlabel("特徴量の重要度")
plt.ylabel("特徴量")
```

![](png2/unnamed-chunk-12-1.png)<!-- -->

- 勾配ブースティングマシンの特徴量の重要度をランダムフォレストと比較すると、いくつかの特徴量が無視されていることがわかる。
- 基本的にはランダムフォレストを先に試したほうが良い。
- 予測時間を短くしたい、チューニングによってギリギリまで性能を高めたいという場合には勾配ブースティングを試す価値がある。
- 勾配ブースティングマシンを大きな問題に試したければ、`xgboost`パッケージの利用を検討したほうが良い。

#### 長所、短所、パラメータ

- 教師あり学習の中で最も強力なモデルの一つ。
- 並列化できないので訓練にかかる時間を短くできない。
- パラメータに影響されやすいので、チューニングを注意深く行う必要がある。
- スケール変換の必要がない、疎なデータには上手く機能しないという点はランダムフォレストと同様。
- 主なパラメータは`n_estimators`と`learning_rate`であるが、ランダムフォレストと異なり`n_estimators`は大きくすれば良いというものではない。大きいほど過学習のリスクが高まる。`n_estimators`をメモリや学習時間との兼ね合いから先に決めておき、`learning_rate`をチューニングするという方法がよくとられる。
- `max_depth`は非常に小さく、5以下に設定される場合が多い。

## アルゴリズム6 カーネル法を用いたサポートベクタマシン

- より複雑なモデルを可能とするため線形モデルを拡張したもの。
- クラス分類にも回帰にも使える(例はクラス分類だけ)。
- 背後にある数学はめっちゃむずい。

### 線形モデルと非線形特徴量

- 線形モデルは特徴量を追加することで柔軟性が増す。
    - 例: 特徴量の交互作用や多項式項を追加する。

過去に扱った、線形分離できない(単純な大小関係だけで判別できない)2クラス分類問題を例に扱う。


```python
from sklearn.datasets import make_blobs
X, y = make_blobs(centers=4, random_state=8)
y = y % 2
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-14-1.png)<!-- -->

このデータは明らかに直線では分離できない。


```python
from sklearn.svm import LinearSVC
linear_svc = LinearSVC().fit(X, y)
mglearn.plots.plot_2d_separator(linear_svc, X)
mglearn.discrete_scatter(X[:, 0], X[:, 1] ,y)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-16-1.png)<!-- -->

特徴量1の二乗を新たな特徴量として加え、データを3次元にしてみる。


```python
X_new = np.hstack([X, X[:, 1:] ** 2])
from mpl_toolkits.mplot3d import Axes3D, axes3d
figure = plt.figure()
ax = Axes3D(figure, elev=-152, azim=-26)
mask = y == 0
ax.scatter(X_new[mask, 0], X_new[mask, 1], X_new[mask, 2], c='b')
ax.scatter(X_new[~mask, 0], X_new[~mask, 1], X_new[~mask, 2], c='r', marker='^')
ax.set_xlabel("特徴量0")
ax.set_ylabel("特徴量1")
ax.set_zlabel("特徴量1の2乗")
```

![](png2/unnamed-chunk-18-1.png)<!-- -->

新しく追加した特徴量方向にはクラス0とクラス1が分離可能になっている。


```python
linear_svm_3d = LinearSVC().fit(X_new, y)
coef, intercept = linear_svm_3d.coef_.ravel(), linear_svm_3d.intercept_
figure = plt.figure()
ax = Axes3D(figure, elev=-152, azim=-26)
xx = np.linspace(X_new[:, 0].min() - 2, X_new[:, 0].max() + 2, 50)
yy = np.linspace(X_new[:, 1].min() - 2, X_new[:, 1].max() + 2, 50)
XX, YY = np.meshgrid(xx, yy)
ZZ = (coef[0] * XX + coef[1] * YY + intercept) / -coef[2]
ax.plot_surface(XX, YY, ZZ, rstride=8, cstride=8, alpha=.3)
ax.scatter(X_new[mask, 0], X_new[mask, 1], X_new[mask, 2], c='b')
ax.scatter(X_new[~mask, 0], X_new[~mask, 1], X_new[~mask, 2], c='r', marker='^')
ax.set_xlabel("特徴量0")
ax.set_ylabel("特徴量1")
ax.set_zlabel("特徴量1の2乗")
```

![](png2/unnamed-chunk-20-1.png)<!-- -->

これを元の空間で見ると、決定境界が直線から曲線になっていることが分かる。


```python
ZZ = YY ** 2
dec = linear_svm_3d.decision_function(np.c_[XX.ravel(), YY.ravel(), ZZ.ravel()])
plt.contourf(XX, YY, dec.reshape(XX.shape), levels=[dec.min(), 0, dec.max()],
             cmap=mglearn.cm2, alpha=.5)
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-22-1.png)<!-- -->

### カーネルトリック

- 上の例では特徴量1の2乗を新しい特徴量として加えることで線形分離可能になったが、通常、**どんな特徴量を追加すれば線形分離可能になるか**は自明ではない。だからといって無闇に特徴量を増やせば計算量が増大してしまう。
- **カーネルトリック**(kernel trick)はこの問題に対応する手法。
- カーネルトリックを使うと実際に特徴量を拡張することなしに、拡張後の空間での特徴量を直接計算できる。この特徴量を直接計算するための関数をカーネルと呼ぶ。例えばガウシアンカーネルと呼ばれるカーネルを使うと、無限次元の空間上で特徴量を計算したのと同じ効果が得られ、しかも計算量はデータ数のみに依存するため、大きくなりすぎる心配がない。
    - テイラー展開すると無限次元への写像になっていることが分かるらしい。
        - cf. [カーネルとは直感的に説明するとなんなのか？ - Qiita](https://qiita.com/keisuke-nakata/items/9d46152ac52dcd2cc92a)
    - ガウシアンカーネルは放射基底関数(radial basis function: RBF)カーネルとも呼ぶ。
- cf. [機械学習におけるカーネル法について - めもめも](http://enakai00.hatenablog.com/entry/2017/10/13/145337)

### SVMを理解する

- SVMは、決定境界の表現にとって個々のデータポイントがどの程度重要かを計算する。
- 基本的には2クラスの境界付近の少数のデータポイントのみが重要となり、これらのデータポイントは**サポートベクタ**と呼ばれる。
- 予測の際は新しいデータポイントとサポートベクタの距離が測定され、これとサポートベクタの重要性を考慮してクラスが決定される。
- ガウシアンカーネルを使用するSVMでは次のガウシアンカーネルを用いて距離を計算する。

$$k_{\mathrm{rbf}}(x_1, x_2) = \exp (- \gamma || x_1-x_2 || ^2) $$

- $x_1$、$x_2$はデータポイントを、$||x_1-x_2||$はユークリッド距離を表し、$\gamma$は制御パラメータである。
    - 注: $\gamma = 1/\sigma^2$として$\sigma$がパラメータとして説明される場合も多い。

SVMによる2クラス分類の結果を示す。


```python
from sklearn.svm import SVC
X, y = mglearn.tools.make_handcrafted_dataset()
svm = SVC(kernel='rbf', C=10, gamma=0.1).fit(X, y)
mglearn.plots.plot_2d_separator(svm, X, eps=.5)
mglearn.discrete_scatter(X[:, 0], X[:, 1], y)
sv = svm.support_vectors_ # サポートベクタをプロット
sv_labels = svm.dual_coef_.ravel() > 0 # クラスラベルのニ値化
mglearn.discrete_scatter(sv[:, 0], sv[:, 1], sv_labels, s=15, markeredgewidth=3)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-24-1.png)<!-- -->

カーネル法を用いたSVMの決定境界は非線形となる。

### SVMパラメータの調整

- `gamma`: ガウシアンカーネルの幅を調整する。
- `C`: 正則化パラメータ。

パラメータと分類の関係を可視化する。


```python
fig, axes = plt.subplots(3, 3, figsize=(15, 10))
for ax, C in zip(axes, [-1, 0, 3]):
  for a, gamma in zip(ax, range(-1, 2)):
    mglearn.plots.plot_svm(log_C=C, log_gamma=gamma, ax=a)
axes[0, 0].legend(["class 0", "class 1", "sv class 0", "sv class 1"],
                  ncol=4, loc=(.9, 1.2))
```

![](png2/unnamed-chunk-26-1.png)<!-- -->

- `gamma`は大きいほど個々のデータポイントを重視するようになり、モデルが複雑になる。
- `C`は小さいほどモデルを制限する点は線形モデルと同様。

**cancer**にRBFカーネルを用いたSVMを適用してみよう。


```python
X_train, X_test, y_train, y_test = train_test_split(
  cancer.data, cancer.target, random_state=0
)
svc = SVC().fit(X_train, y_train)
print("訓練セットの精度: {:.3f}".format(svc.score(X_train, y_train)))
 ## 訓練セットの精度: 1.000
print("テストセットの精度: {:.3f}".format(svc.score(X_test, y_test)))
 ## テストセットの精度: 0.629
```

- 訓練セットの精度が1であり、テストセットの精度が低く、過学習している。
- SVMはパラメータとデータのスケールに敏感に反応する。
    - 特にデータのスケールは全ての特徴量で揃っている必要がある。

データのスケールを確認するために、個々の特徴量に対して対数スケールで箱ひげ図を作成する。


```python
plt.boxplot(X_train, sym='+')
plt.ylim(10**-1, 10**4)
plt.xlabel("特徴量のインデックス")
plt.ylabel("特徴量の大きさ")
plt.yscale("log")
```

![](png2/unnamed-chunk-29-1.png)<!-- -->

### SVMのためのデータの前処理

- **cancer**は前掲のようにデータのスケールが特徴量により著しく異なる。
- 前処理のちゃんとしたやり方は後で説明するので、ここではとりあえず手作業でスケールを揃える。


```python
# 最小値の計算
min_on_training = X_train.min(axis=0)
# レンジ = 最大値 - 最小値を計算
range_on_training = (X_train - min_on_training).max(axis=0)
# 最小値を引いてからレンジで割ることで、min=0、max=1に変換される
X_train_scaled = (X_train - min_on_training) / range_on_training
print("スケール済み特徴量の最小値: {}".format(X_train_scaled.min(axis=0)))
 ## スケール済み特徴量の最小値: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 ##  0. 0. 0. 0. 0. 0.]
print("スケール済み特徴量の最大値: {}".format(X_train_scaled.max(axis=0)))
 ## スケール済み特徴量の最大値: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 ##  1. 1. 1. 1. 1. 1.]
```

テストセットについても同様の変換を行うが、最小値とレンジは訓練セットのものを使う。


```python
X_test_scaled = (X_test - min_on_training) / range_on_training
```

したがって、テストセットのレンジは0〜1ではない。この点については第4章で解説される（らしい）。


```python
print("テストセットの最小値: {}".format(X_test_scaled.min(axis=0)))
 ## テストセットの最小値: [ 0.03540158  0.04190871  0.02895446  0.01497349  0.14260888  0.04999658
 ##   0.          0.          0.07222222  0.00589722  0.00105015 -0.00057494
 ##   0.00067851 -0.0007963   0.05148726  0.01434497  0.          0.
 ##   0.04195752  0.01113138  0.03678406  0.01252665  0.03366702  0.01400904
 ##   0.08531995  0.01833687  0.          0.          0.00749064  0.02367834]
print("テストセットの最大値: {}".format(X_test_scaled.max(axis=0)))
 ## テストセットの最大値: [0.76809125 1.22697095 0.75813696 0.64750795 1.20310633 1.11643038
 ##  0.99906279 0.90606362 0.93232323 0.94903117 0.45573058 0.72623944
 ##  0.48593507 0.31641282 1.36082713 1.2784499  0.36313131 0.77476795
 ##  1.32643996 0.72672498 0.82106012 0.87553305 0.77887345 0.67803775
 ##  0.78603975 0.87843331 0.93450479 1.0024113  0.76384782 0.58743277]
```


スケール済みの特徴量を使ってSVCを行う。


```python
svc = SVC().fit(X_train_scaled, y_train)
print("訓練セットに対する精度: {:.3f}".format(svc.score(X_train_scaled, y_train)))
 ## 訓練セットに対する精度: 0.948
print("テストセットに対する精度: {:.3f}".format(svc.score(X_test_scaled, y_test)))
 ## テストセットに対する精度: 0.951
```

スケール前に比べるとかなりの改善が見られる。さらにパラメータを少し調整すると、さらに良くなる。


```python
svc = SVC(C=1000).fit(X_train_scaled, y_train)
print("訓練セットに対する精度: {:.3f}".format(svc.score(X_train_scaled, y_train)))
 ## 訓練セットに対する精度: 0.988
print("テストセットに対する精度: {:.3f}".format(svc.score(X_test_scaled, y_test)))
 ## テストセットに対する精度: 0.972
```

### 利点、欠点、パラメータ

- いろいろなデータに対して上手く機能する強力なモデルである。
    - データに僅かな特徴量しかなくても決定境界が引ける。
    - 低次元のデータでも高次元のデータでも機能する。
    - サンプルの個数が多いと上手くいかないことがある。
        - 10,000サンプルくらいまではいけるが、100,000サンプルくらいになるとメモリや速度面から厳しくなってくる。
- データの前処理とパラメータのチューニングは必須といえる。
- モデルについて決定木のように誰にでも分かる形で説明するのは難しい。
- パラメータは正則化パラメータ`C`とカーネル固有のパラメータがある。
    - RBFカーネル以外のカーネルを使うこともできる。
    - RBFカーネルにおける`C`と`gamma`は強く相関するため同時いチューニングする必要がある。

## ニューラルネットワーク(ディープラーニング)

- 最近流行りの例のアレ。
- 実際のアルゴリズムの多くは特定の用途向けに注意深く作られたものになっている。
- ディープラーニングの中でも割と簡単な**多層パーセプトロン**(multilayer perceptron: MLP)を例にする。

### ニューラルネットワークモデル

- MLPは線形モデルを一般化して複数ステージで計算するもの。
- 線形回帰は重み付きの和を計算している。
<img src="png2/nuro1.png" width="200px">

- MLPは重み付き和の計算を繰り返す。中間ステップを**隠れユニット**と呼ぶ。
<img src="png2/nuro2.png" width="300px">

- 重み付き和を単に連続して計算するのは、1つの重み付き和を計算するのと等価なので、もう少し工夫しないと線形モデルと変わらない。

- ざっくりとした説明: 
    - 入力から隠れ層への重みを${\bf W}_1$、隠れ層から出力への重みを${\bf W}_2$と行列で表す。
    - このとき、隠れ層の値${\bf h}$は、入力${\bf x}$と重みの積として${\bf h}={\bf W_1x}$として表せる。
    - 同様に、$y={\bf W_2h}$。
    - 全体は$y={\bf W_2 W_1 x}$のように表現できるが、ここで${\bf W_3}={\bf W_2W_1}$を考えることができるので、結局2つの重み付き和を計算するのは1つの重み付き和を計算するのと変わらない。
    - 同様に、層を何層に増やしても途中の計算が線形である限り、必ず1層の等価な重み付き和が存在する。


- もう少しの工夫 = 重み付き和の計算結果に**非線形関数**を適用する。
    - relu(rectified linear unit: 正規化線形関数)
        - ゼロ以下の値を切り捨てる。
    - tanh(hyperbolic tangent: 双極正接関数)
        - 小さい値は-1、大きい値は+1に飽和。
- この工夫によって線形モデルよりはるかに複雑なモデルを学習可能となる。
    - 重み付き和を出力する際に変換を行う関数は**活性化関数**(activation function)とも呼ばれる。
    
relu関数とtanh関数を示す。relu関数は0との間でmaxを取れば良い。tanh関数はNumPyに実装されている。


```python
line = np.linspace(-3, 3, 100)
plt.plot(line, np.tanh(line), label="tanh")
plt.plot(line, np.maximum(line, 0), label="relu")
plt.legend(loc="best")
plt.xlabel("x")
plt.ylabel("relu(x), tanh(x)")
```

![](png2/unnamed-chunk-38-1.png)<!-- -->

- 隠れ層を追加したり、隠れ層のノード数を増やしたりしてより複雑なニューラルネットを作成することができる。ノード数は時には10,000にもなる。
- このような計算層を多数持つニューラルネットから**ディープラーニング**という言葉が生まれた。
<img src="png2/nuro3.png" width="400px">


### ニューラルネットワークのチューニング

**two_moons**データセットを用いる。


```python
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=.25, random_state=3)
X_train, X_test, y_train, y_test = train_test_split(
  X, y, stratify=y, random_state=42
)
mlp = MLPClassifier(solver='lbfgs', random_state=0).fit(X_train, y_train)
mglearn.plots.plot_2d_separator(mlp, X_train, fill=True, alpha=.3)
mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-41-1.png)<!-- -->

- デフォルトではMLPは100のノードからなる単一の隠れ層を持つが、これは小さなデータセットに対しては大きすぎるので10に減らしてみる。


```python
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[10])
mlp.fit(X_train, y_train)
mglearn.plots.plot_2d_separator(mlp, X_train, fill=True, alpha=.3)
mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-43-1.png)<!-- -->

- 上記の例で境界がギザギザなのは、デフォルトで活性化関数がrelu関数のため。
- 層を増やしたり、活性化関数にtanhを用いることで境界を滑らかにできる。

まず隠れ層を1層ふやしてみる。


```python
# 10ユニットの隠れ層を2つ使う
mlp = MLPClassifier(solver='lbfgs', random_state=0, hidden_layer_sizes=[10, 10])
mlp.fit(X_train, y_train)
mglearn.plots.plot_2d_separator(mlp, X_train, fill=True, alpha=.3)
mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-45-1.png)<!-- -->

さらに活性化関数にtanhを指定する。


```python
# 10ユニットの隠れ層を2つ使う+活性化関数にtanh
mlp = MLPClassifier(solver='lbfgs', activation='tanh',
                    random_state=0, hidden_layer_sizes=[10, 10])
mlp.fit(X_train, y_train)
mglearn.plots.plot_2d_separator(mlp, X_train, fill=True, alpha=.3)
mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train)
plt.xlabel("特徴量0")
plt.ylabel("特徴量1")
```

![](png2/unnamed-chunk-47-1.png)<!-- -->

- ニューラルネットワークにはまだまだパラメータがある。
- 重みに対してL2正則化を行うことができる。デフォルトでは正則化は非常に弱い。

以下は10ノードと100ノードの2層の隠れ層を持つニューラルネットワークに対し、L2正則化の程度を調整するパラメータalphaを変えた効果を示している。


```python
fig, axes = plt.subplots(2, 4, figsize=(20, 8))
for axx, n_hidden_nodes in zip(axes, [10, 100]):
  for ax, alpha in zip(axx, [0.0001, 0.01, 0.1, 1]):
    mlp = MLPClassifier(solver='lbfgs', random_state=0,
                        hidden_layer_sizes=[n_hidden_nodes, n_hidden_nodes],
                        alpha=alpha)
    mlp.fit(X_train, y_train)
    mglearn.plots.plot_2d_separator(mlp, X_train, fill=True, alpha=.3, ax=ax)
    mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train ,ax=ax)
    ax.set_title("隠れ層=[{}, {}]\nalpha={:.4f}".format(
                 n_hidden_nodes, n_hidden_nodes, alpha))
```

![](png2/unnamed-chunk-49-1.png)<!-- -->

- ニューラルネットワークは重みの初期値を乱数で決めるが、この影響は小さいネットワークでは大きく現れることがある。


```python
fig, axes = plt.subplots(2, 4, figsize=(20, 8))
for i, ax in enumerate(axes.ravel()):
  mlp = MLPClassifier(solver='lbfgs', random_state=i,
                      hidden_layer_sizes=[100, 100])
  mlp.fit(X_train, y_train)
  mglearn.plots.plot_2d_separator(mlp, X_train, fill=True, alpha=.3, ax=ax)
  mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train, ax=ax)
```

![](png2/unnamed-chunk-51-1.png)<!-- -->

次に、実データとしてcancerを使ってニューラルネットワークを適用してみる。

cancerはデータセットのレンジが非常に幅広いデータである。


```python
print(cancer.data.max(axis=0)) # 各データセットの最大値
 ## [2.811e+01 3.928e+01 1.885e+02 2.501e+03 1.634e-01 3.454e-01 4.268e-01
 ##  2.012e-01 3.040e-01 9.744e-02 2.873e+00 4.885e+00 2.198e+01 5.422e+02
 ##  3.113e-02 1.354e-01 3.960e-01 5.279e-02 7.895e-02 2.984e-02 3.604e+01
 ##  4.954e+01 2.512e+02 4.254e+03 2.226e-01 1.058e+00 1.252e+00 2.910e-01
 ##  6.638e-01 2.075e-01]
```

まずはデータセットそのままでニューラルネットワークを適用する。


```python
X_train, X_test, y_train, y_test = train_test_split(
  cancer.data, cancer.target, random_state=0
)
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)
print("訓練セットの精度: {:.2f}".format(mlp.score(X_train, y_train)))
 ## 訓練セットの精度: 0.91
print("テストセットの精度: {:.2f}".format(mlp.score(X_test, y_test)))
 ## テストセットの精度: 0.88
```

精度は良いもののさほどではない。MLPはデータのスケールが同じくらいであることが望ましい。また、**平均が0で分散が1**であれば理想的である。そのような変換をここでは手作業で行う(`StandardScaler`を使えばもっと簡単にできるが、これは後に説明される)。


```python
mean_on_train = X_train.mean(axis=0) # 各データセットの平均値
std_on_train = X_train.std(axis=0) # 各データセットの標準偏差
# 平均を引いてスケーリングする
X_train_scaled = (X_train - mean_on_train) / std_on_train
X_test_scaled = (X_test - mean_on_train) / std_on_train
# MLPを適用
mlp = MLPClassifier(random_state=0)
mlp.fit(X_train_scaled, y_train)
 ## /usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 ##   % self.max_iter, ConvergenceWarning)
print("訓練セットの精度: {:.3f}".format(mlp.score(X_train_scaled, y_train)))
 ## 訓練セットの精度: 0.991
print("テストセットの精度: {:.3f}".format(mlp.score(X_test_scaled, y_test)))
 ## テストセットの精度: 0.965
```

これで精度はグッと良くなったが、収束に関する警告が出ている。繰り返し数が不足しているので、`max_iter`パラメータを通じて繰り返し数を増やす。


```python
mlp = MLPClassifier(max_iter=1000, random_state=0)
mlp.fit(X_train_scaled, y_train)
print("訓練セットの精度: {:.3f}".format(mlp.score(X_train_scaled, y_train)))
 ## 訓練セットの精度: 0.993
print("テストセットの精度: {:.3f}".format(mlp.score(X_test_scaled, y_test)))
 ## テストセットの精度: 0.972
```

訓練セットに対する精度は上がったが、汎化性能があまり変化しない。パラメータalphaを大きくして、正則化を強くし、モデルを単純にするともっと汎化性能が上がるかもしれない。


```python
mlp = MLPClassifier(max_iter=1000, alpha=1, random_state=0)
mlp.fit(X_train_scaled, y_train)
print("訓練セットの精度: {:.3f}".format(mlp.score(X_train_scaled, y_train)))
 ## 訓練セットの精度: 0.988
print("テストセットの精度: {:.3f}".format(mlp.score(X_test_scaled, y_test)))
 ## テストセットの精度: 0.972
```

ニューラルネットワークの解析は線形モデルや決定木に比べると難しい。

隠れ層における重みを可視化するという手があるので以下に示す。


```python
plt.figure(figsize=(20, 5))
plt.imshow(mlp.coefs_[0], interpolation='none', cmap='viridis')
plt.yticks(range(30), cancer.feature_names)
plt.xlabel("重み行列の列")
plt.ylabel("特徴量")
plt.colorbar()
```

![](png2/unnamed-chunk-58-1.png)<!-- -->

**隠れ層の全てのノードで重みが少ない特徴量**は、少なくともこのモデルにおいて重要ではないことが予想される。本当に重要でないのか、あるいは特徴量がニューラルネットワークが利用可能な形で表現されていなかったかのどちらかだ。

- ニューラルネットワークをより柔軟に、より大きなデータに適用したければディープラーニングライブラリを試すと良い。
- **keras**、**lasagne**、**tensor-flow**が有名
    - theanoやtensor-flowはディープラーニングのライブラリで、kerasやlasagneはそれらライブラリ上で動作するディープラーニングのライブラリ。
    - lasagneはラザニアの複数形でラザーニェとか読むらしい。テキストのlasagnaは誤字っぽい。Kaggleでよく使われてるとかいう記事が出るけど、2015年あたり以降流行ってないのかも。ラザニアのレシピがやたらヒットして検索しにくい。

### 長所、短所、パラメータ

- 大量のデータを使って非常に複雑なモデルを構築できること。
- 時間とデータを費やし十分にパラメータ調整を行えば回帰でも分類でも他のアルゴリズムに勝てる可能性がある。
- 訓練には時間がかかる。
- それぞれの特徴量のスケールが近くないと上手く動かない。
- パラメータのチューニングはそれ自体が技術となる程度に複雑で奥が深い。

#### ニューラルネットワークの複雑さ推定

- 隠れ層と層あたりのノード数は最も重要なパラメータ。
    - 層は1つか2つから初め、後で増やしていくとよい。
    - ノード数は入力と同じくらいが多いが、数千より大きくすることは少ない。
- 学習の対象である重みの数は複雑さの指標の一つとなる。
    - 例1: 100の特徴量、隠れ層(100ノード)、出力1
        - 入力から隠れ層: 100行100列の行列が必要 => 重みの数は10,000
        - 隠れ層から出力: 100列1行の行列が必要(入力が縦ベクトルの場合) => 重みの数は100
        - 合計10,100の重みを学習する必要がある。
    - 例2: 100の特徴量、隠れ層×2(1,000ノード2層)、出力1
        - 入力から隠れ層1: 100行1000列の行列が必要
- パラメータ調整の定石
    - まず大きめのネットワークを作って過学習させる
        - そもそも訓練データを学習できるのかを確認
    - ネットワークを小さくしたり、正則化パラメータを調整して汎化性能を上げる
- どのようにモデルを学習させるか？
    - 言い換えると…→どのように損失関数=誤差の値を最小化するか？
    - [An overview of gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/index.html#visualizationofalgorithms)のグラフを見るとイメージがつかみやすいと思います。
    - 初心者は'adam'か'lbfgs'を使っておくとよい。
- fitはモデルをリセットする…？
    - これは`MLPClassifier`をデフォルトでインスタンス化した場合の話で、パラメータ`warm_start`に`True`を指定すると前回の学習を引き継げるようになるようです。
    - cf. [ニューラルネットワークのパラメータ設定方法(scikit-learnのMLPClassifier)](https://spjai.com/neural-network-parameter/)
